## 전처리: 자연어의 노이즈 제거

### 노이즈 유형 (1) 문장부호 : Hi, my name is john.

In [28]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")
        
    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [",", "?", "!", ","]))

Hi  ,   my name is john.


### 노이즈 유형 (2) 대소문자 : First, open the first chapter.

In [29]:
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


In [30]:
sentence = "First, open the first chapter."

print(sentence.upper())

FIRST, OPEN THE FIRST CHAPTER.


### 노이즈 유형 (3) 특수문자 : He is a ten-year-old boy.

In [31]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


In [32]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
"""

def cleaning_text(text, punc, regex):
    # 노이즈 유형(1) 문장부호 공백 추가
    for p in punc:
        text = text.replace(p, " " +p+" ")
        
    # 노이즈 유형(2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()
    
    return text

print(cleaning_text(corpus, [".",".","!","?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk  .   
but my teacher had been with me several weeks before i understood that everything has a name  .  
one day, we walked down the path to the well house, attracted by the fragrance of the honeysuckle with which it was covered  .   
some one was drawing water and my teacher placed my hand under the spout  .   
as the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly  .   
i stood still, my whole attention fixed upon the motions of her fingers  .   
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me  .   
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand  .   
that living word awakened my soul, gave it light, hope, joy, set it free !  


## 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?
### 단어의 희소 표현과 분산 표현

분산표현(distributed representation)과 희소표현(Sparse representation)은 단어를 벡터로 표현하겠다는 점에서 동일합니다.  
하지만 희소표현의 경우 벡터의 각 차원마다 단어의 특정 의미 속성을 대응시키는 방식입니다.  
ex) [성별, 연령, 키, 몸무게]  
이런 방식을 사용하면 메모리의 연산량이 낭비될 뿐만 아니라 단어들 간의 의미적 유사도를 구할 수 없다는 한계가 발생합니다.  
  
그래서 Embedding layer를 사용해 각 단어가 몇 차원의 속성을 가질지 정의하는 방식으로 단어의 분산표현(distributed representation)을 구현하는 방식을 주로 사용합니다.  
ex) 100개의 단어를 256차원의 속성으로 표현  
embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)  
위 단어의 분산 표현에는 우리가 일일이 정의할 수 없는 어떤 추상적인 속성들이 256차원 안에 골고루 분산되어 표현됩니다. 적절히 훈련된 분산 표현 모델을 통해 우리는 단어 간의 의미 유사도를 계산하거나, 이를 feature로 삼아 복잡한 자연어처리 모델을 훈련시킬 수 있게 됩니다.

### 단어 사전 구성과 활용의 문제
위와 같이 단어 사전을 만들었다고 하더라도 컴퓨터가 엉뚱한 단어로 해석하거나 사전에서 해당 단어를 찾지 못할 수 있습니다. 이는 컴퓨터가 문장을 단어 단위로 정확하게 끊어 읽지 못하기 때문에 발생합니다.

## 토큰화: 그녀는? 그녀+는?
### 공백 기반 토큰화
자연어의 노이즈를 제거하는 방법 중 하나로 우리는 Hi, 를 Hi와 ,로 나누기 위해 문장부호 양옆에 공백을 추가해 주었습니다.

In [33]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

위와 같은 방식을 사용하면 사실상 같은 단어인 days 와 day 가 구분되어 따로 저장이 되는 정도의 문제는 있겠지만, s 를 모두 한 칸 띌 수도 없는 노릇이기에 불가피한 손실로 취급합니다

### 형태소 기반 토큰화
형태소의 정의: (명사) 뜻을 가진 가장 작은 말의 단위.  
    예를 들어, 오늘도 공부만 한다 라는 문장이 있다면, 오늘, 도, 공부, 만, 한다 로 쪼개지는 것이 바로 형태소입니다. 한국어는 이를 활용해 토큰화를 진행할 수 있습니다.

In [34]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

In [35]:
import khaiii

api = khaiii.KhaiiiApi()
api.open()

In [36]:
# Khaiii를 konlpy tokenizer처럼 사용하기 위한 wrapper class입니다. 

class Khaiii():
    def pos(self, phrase, flatten=True, join=False):
        """POS tagger.

        :param flatten: If False, preserves eojeols.
        :param join: If True, returns joined sets of morph and tag.

        """
        sentences = phrase.split('\n')
        morphemes = []
        if not sentences:
            return morphemes

        for sentence in sentences:
            for word in api.analyze(sentence):
                result = [(m.lex, m.tag) for m in word.morphs]
                if join:
                    result = ['{}/{}'.format(m.lex, m.tag) for m in word.morphs]

                morphemes.append(result)

        if flatten:
            return sum(morphemes, [])

        return morphemes

In [37]:
tokenizer_list = [Hannanum(),Kkma(),Mecab(),Okt(),Khaiii()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Mecab] 
[('코로나', 'NNP'), ('바이러스', 'NNG'), ('는', 'JX'), ('

위와 같이 토크나이저들은 각자가 장단점을 가지고 있습니다.

### 사전에 없는 단어의 문제
공백 기반이나, 형태소 기반의 토큰화 기법들은 모두 의미를 가지는 단위로 토큰을 생성합니다. 이 기법의 경우, 데이터에 포함되는 모든 단어를 처리할 수는 없기 때문에 자주 등장한 상위 N개의 단어만 사용하고 나머지는 <unk>같은 특수한 토큰(Unknown Token)으로 치환해버립니다. 하지만 이것은 종종 큰 문제를 야기합니다.

코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.  
  
    
->  
  
    
<unk>는 2019년 12월 중국 <unk>에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

만약 위 문장을 영문으로 번역해야 한다면 어떨까요? 핵심인 단어 코로나바이러스와 우한을 모른다면 제대로 해낼 수 있을 리가 없습니다. 이를 OOV(Out-Of-Vocabulary) 문제라고 합니다. 이를 해결하고자 하는 많은 시도들이 있었습니다.

## 단어의 조각들, Wordpiece Model
두 단어 preview와 predict를 보면 접두어인 pre가 공통됩니다. pre는 "미리", "이전의"와 연관된 의미를 가지고 있습니다. 이처럼 컴퓨터 또한 두 단어를 따로 보는 것이 아니라 pre+view와 pre+dict로 본다면 학습을 더 잘 할수도 있을 것입니다.  
ㅇ이와 같이 하나의 단어를 여러 Subword의 집합으로 보는 방법이 WPM입니다. 이 원리를 알기 전에 먼저 알아야 할 것이 Byte Pair Encoding(BPE)입니다.

### Byte Pair Encoding(BPE)
1994년 고안된 BPE 알고리즘은 데이터 압출을 위해 탄샌했습니다. 데이터에서 가장 많이 들장하는 바이트 쌍(Byte Pair)을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작합니다.  
  
**사용예시**  
aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.  
→  
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.  
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.  
→  
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.  
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면  
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.  
→  
XdXac  
Z=aa  
Y=ab  
X=ZY       # 압축이 완료되었습니다!

이를 토큰화에 적용하자고 제안한 것은 2015년이었습니다. 자주 들장하는 문자 쌍을 합치면 접두어나 접미어의 의미를 캐치할 수 있고 처음 등장하는 단어는 문자(알파벳)들의 조합으로 나타내어 OOV 문제를 해결할 수 있습니다.

In [ ]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

만일 lowest라는 처음 보는 단어가 등장하더라도, 위 알고리즘을 따르면 어느 정도 의미가 파악된 low와 est의 결합으로 표현할 수 있습니다.  
Embedding 레이어는 단어의 개수 x Embedding 차원 수 의 Weight를 생성하기 때문에 단어의 개수가 줄어드는 것은 곧 메모리의 절약으로 이어집니다. 많은 데이터가 곧 정확도로 이어지기 때문에 이는 큰 의미가 있는 것입니다.  
하지만 이러한 방식에도 문제가 있습니다. **[i, am, a, b, o, y, a, n, d, you, are, a, gir, l]**과 같은 방식으로 단어장을 만들 수 있기 때문입니다.

### Wordpiece Model(WPM)
구글에서 BPE를 변형해 제안한 WPM 알고리즘은 이러한 문제를 개선할 수 있습니다. WPM은 BPE에 대해 두 가지 차별성을 가집니다.  
1. 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.  
2. 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.  

WPM을 통해서 어떤 언어에도 OOV 발생 우려 없이 토크나이징을 할 수 있지만 제대로 된 단어의 분산 표현을 얻을 수는 없습니다.  
한국어라면 **자동차**에서 **_자동 / 차**로 분리됐을 때 **"차"**가 달리는 차인지, 마시는 차인지 컴퓨터는 구분지을 수 없기 때문입니다.

## 토큰에게 의미를 부여하기
토큰들에 랜덤하게 부여된 실수에 의미를 부여하는 많은 알고리즘이 있습니다. 이 중 3가지만 알아보겠습니다.

### Word2Vec
Word2Vec은 "단어를 벡터로 만든다"는 이름을 가지고 있습니다. 문장의 각 단어 즉, 동시에 등장하는 단어끼리는 연관성이 있다는 아이디어로 시작된 알고리즘입니다.  
"난 오늘 술을 마셨어" 라는 문장에서 "술"과 "마셨어" 간에는 어떤 연관성을 찾을 수 있습니다.

### FastText
Word2Vec은 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있습니다. FastText는 이를 해결하기 위해 BPE와 비슷한 아이디어를 적용했습니다.  
FastText는 SGNS 방법에 기반을 두고 있으며 SGNS와는 달리 텍스트의 최소 단위가 '어휘'가 아닌 '어휘를 구성하는 글자 n_gram 단위'로 한 수준 내린 것이 특징입니다.  
ex) 어휘: 'where', ngram: 5 (n=3)  
<wh, whe, her, ere, er>  
어휘의 시작과 끝을 뜻하는 기호 <,>  
이와 같은 방식을 적용하면 어휘의 구문적(syntactic) 변화 규칙을 잘 잡아낼 수 있으며 더 적은 양의 텍스트 데이터 상에서도 효과적으로 작동하고, 학습 시 관찰하지 못한 어휘에 대해서도 양질의 임베딩 벡터를 만들어낼 수 있습니다.

### ELMo - the 1st Comtextualized Word Embedding
위의 Word Embedding 알고리즘들은 동음이의어를 처리할 수 없습니다.  
* 이렇게나 탐스럽고 먹음직스러운 **사과**를 보셨나요?
* 저의 간절한 **사과**를 받아주시기 바랍니다.  
이 두 문장에서 '사과'의 의미는 각각 다르지만 Word2Vec과 FastText의 워드 벡터 값은 동일할 수 밖에 없습니다.  

Context sensitive Gramaar를 따르는 자연어를 이해하려면 문맥(context)의 활용이 필수적입니다.  
'사과'의 경우 첫 문장에서는 탐스럽고, 먹음직스러운이 될 것이며 두번째 문장에서는 간절한이 될 것입니다.  
즉, 단어의 의미 벡터를 구하기 위해서는 그 단어만 필요한 것이 아니라 그 단어가 놓인 주변 단어 배치의 맥락이 함께 고려되는 Word Embedding이 필요한 것입니다. 이런 개념을 맥락화된 어휘 임베딩(Contextualized Word Embedding)이라고 합니다.